In [15]:
basePath = '../calibration_data/western_battery_1/'
labelsRelativePath = basePath + 'labels.yaml'
imagesRelativePath = basePath + 'calibration_scene_images'
matchesRelativePath = basePath + 'calibration_scene_matches'
reconstructionRelativePath = basePath + 'calibration_scene_reconstruction'

import os
basePath = os.path.abspath('')
labelsPath = os.path.join(basePath, labelsRelativePath)
matchesPath = os.path.join(basePath, matchesRelativePath)
sfmDataRelativePath = os.path.join(matchesRelativePath,'sfm_data.json')
sfmDataPath = os.path.join(matchesPath,'sfm_data.json')
reconstructionPath = os.path.join(basePath, reconstructionRelativePath)


import json
import yaml
import itertools
import numpy as np
from collections import OrderedDict 

assert os.path.isfile(labelsPath), "Cant find your label yaml file"

In [16]:
with open(labelsPath) as file:
    labelsDict = yaml.full_load(file)

labelsList = []
viewList = []
for frameNumber in list(labelsDict.keys()):
    frameLabels = labelsDict[frameNumber]
    for label in frameLabels:
        newLabel = label.copy()
        newLabel['class'] = newLabel['class'][0]+'-'+frameNumber
        labelsList.append(newLabel)
        if newLabel['view'] not in viewList:
            viewList.append(newLabel['view'])

viewList.sort()
viewLabels = [OrderedDict() for _ in range(len(viewList))] #Note: [[]]*len(viewList) doesnt work with .append
count = 0
for label in labelsList:
    newKey = label['class']
    viewLabels[label['view']][newKey] = label['location']

pairCombinations = list(itertools.combinations(list(range(len(viewLabels))),2))
pairMatches = [[] for _ in range(len(pairCombinations))]
for i, pairCombination in enumerate(pairCombinations):
    halfMatchesA = list(viewLabels[pairCombination[0]].keys())
    halfMatchesB = list(viewLabels[pairCombination[1]].keys())
    for halfMatch in halfMatchesA:
        if halfMatch in halfMatchesB:
            pair = (halfMatchesA.index(halfMatch),halfMatchesB.index(halfMatch))
            if pair[::-1] not in pairMatches[i]:
                pairMatches[i].append(pair)
            

In [17]:
# create the sfm_data.json file
# fuction documentation: https://openmvg.readthedocs.io/en/latest/software/SfM/SfMInit_ImageListing/
!openMVG_main_SfMInit_ImageListing \
-i {imagesRelativePath} \
-d ../openmvg/openMVG/src/openMVG/exif/sensor_width_database/sensor_width_camera_database.txt \
-o {matchesRelativePath} \
-g 1 # 1 is the defaut, all views are one camera. 0 gives each view its own intrinsics

# get the file names
with open(sfmDataRelativePath) as json_file:
    data = json.load(json_file)
    viewNames = [None for _ in range(len(data['views']))]
    for view in data['views']:
        viewNames[view['key']] = view['value']['ptr_wrapper']['data']['filename'].split('.')[0]
        
# dump feature files
for i, name in enumerate(viewNames):
    featureFileName = os.path.join(matchesPath,name+'.feat')
    keypoints = np.array(list(viewLabels[i].values()))
    features = np.pad(keypoints, pad_width=[[0,0],[0,2]], mode='constant', constant_values=0)
    np.savetxt(featureFileName, features, fmt='%.6g')
    
# write match file
with open(os.path.join(matchesPath,'matches.e.txt'),'w') as file:
    for i, pairCombination in enumerate(pairCombinations):
        file.write(f'{pairCombination[0]} {pairCombination[1]}\n')
        file.write(f'{len(pairMatches[i])}\n')
        for pairMatch in pairMatches[i]:
            file.write(f'{pairMatch[0]} {pairMatch[1]}\n')

 You called : 
openMVG_main_SfMInit_ImageListing
--imageDirectory ../calibration_data/western_battery_1/calibration_scene_images
--sensorWidthDatabase ../openmvg/openMVG/src/openMVG/exif/sensor_width_database/sensor_width_camera_database.txt
--outputDirectory ../calibration_data/western_battery_1/calibration_scene_matches
--focal -1
--intrinsics 
--camera_model 3
--group_camera_model 1

- Image listing -
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

SfMInit_ImageListing report:
listed #File(s): 4
usable #File(s) listed in sfm_data: 4
usable #Intrinsic(s) listed in sfm_data: 0


## NOTE: This is where we run into issues for the first time

In [18]:
# fuction documentation: https://openmvg.readthedocs.io/en/latest/software/SfM/GlobalSfM/
!openMVG_main_GlobalSfM -i {sfmDataRelativePath} -m {matchesRelativePath} -o {reconstructionRelativePath}

#!openMVG_main_IncrementalSfM -i {sfmDataRelativePath} -m {matchesRelativePath} -o {reconstructionRelativePath}


-----------------------------------------------------------
Global Structure from Motion:
-----------------------------------------------------------
Open Source implementation of the paper:
"Global Fusion of Relative Motions for Robust, Accurate and Scalable Structure from Motion."
Pierre Moulon, Pascal Monasse and Renaud Marlet.  ICCV 2013.
------------------------------------------------------------

- Features Loading -
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

CleanGraph_KeepLargestBiEdge_Nodes():: => connected Component: 1
Connected component of size: 4

- Relative pose computation -
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
Relative motion computation took: 0(ms)
GlobalSfM:: Rotation Averaging failure!
